In [22]:
import numpy as np
import pandas as pd

# install cmake, libomp
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from matplotlib import pyplot

In [23]:
# Get data-set
main_df = pd.read_csv('./movie_clean_bc.csv', encoding='cp949')

In [24]:
#main_df.describe()
#main_df.info()
#main_df.shape[0]
#main_df['15세이상관람가'].unique()
df = main_df 
drop_flag = False

In [25]:
# 상영 전 평론가, 관객의 점수는 지표로 사용할 수 없다. (Drop)
if drop_flag == False:
    df = df.drop(['비평가 점수', '관객 평가(네티즌평가)'], axis=1)
    drop_flag = True

In [26]:
y = [idx for idx in range(0, df.shape[0])]
# Test-Set 분리
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.5, random_state=10)
x_train.shape, x_test.shape

((1644, 34), (1644, 34))

In [27]:
y = np.log1p(x_train['총관객수'])
y.head(), y.shape

(414     12.825845
 3091    10.031661
 2459    10.045768
 1911     9.775654
 2998    13.007401
 Name: 총관객수, dtype: float64,
 (1644,))

In [28]:
# Test set도 결과도 log 취함
y_vali = np.log1p(x_test['총관객수'])

In [29]:
x = x_train.drop('총관객수', axis=1)
x_vali = x_test.drop('총관객수', axis=1)
x_vali.head(), x_vali.shape

(      계절성여부  상영시간(분)  스크린수  전체관람가  12세이상관람가  15세이상관람가      배우파워1      배우파워2  \
 413       0      110   299      0         0         1  2199359.0  1066765.0   
 290       0      103   182      0         0         1   520753.0        1.0   
 860       1       95   167      0         1         0        1.0        1.0   
 2952      0      106   143      0         0         1  2026574.0   137340.0   
 1835      0      124   356      1         0         0   243094.0   471310.0   
 
         감독파워  다양성(독립)영화  ...  범죄  미스터리  뮤지컬  가족  서부극(웨스턴)  공연  성인물(에로)  기타  \
 413    10036          0  ...   0     0    0   0         0   0        0   0   
 290   101879          0  ...   0     0    0   0         0   0        0   0   
 860        1          1  ...   0     0    0   0         0   0        0   0   
 2952   20413          1  ...   0     0    0   0         0   0        0   0   
 1835  742475          0  ...   0     0    0   0         0   0        0   0   
 
       다국적  배급사파워1  
 413     1      29  


In [30]:
# K-fold로 10개의 generator 생성
kf = KFold(n_splits =  10, shuffle = True, random_state = 96)
# Make Model
xgb = XGBRegressor(random_state = 256)

In [31]:
rmse_list = []
xgb_pred = np.zeros((x_train.shape[0]))

# K(10)-fold validation Loop
for tr_idx, val_idx in kf.split(x, y) :
    tr_x, tr_y = x.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]
    
    xgb.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in xgb.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in xgb.predict(x_vali)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    
    rmse_list.append(rmse)
    xgb_pred += (sub_pred / 10)

* regression Model accuracy

In [32]:
"RMSE (Root mean square error ): {}".format(np.mean(rmse_list))

'RMSE (Root mean square error ): 1419981.7625672037'